In [1]:
 !pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"l4e1owek","key":"a31d8529a9ceb19f66a1f35ce6044f83"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c ml-intensive-yandex-academy-spring-2024 -p data

 99% 980M/988M [00:09<00:00, 72.0MB/s]
100% 988M/988M [00:09<00:00, 113MB/s] 


In [6]:
import zipfile
import os

# Путь к вашему архиву
zip_file_path = '/content/data/ml-intensive-yandex-academy-spring-2024.zip'

# Папка, куда вы хотите извлечь содержимое архива
extract_folder = '/content/'

# Создаем папку для извлеченных файлов, если она еще не существует
os.makedirs(extract_folder, exist_ok=True)

# Разархивируем архив
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Архив успешно разархивирован.")

Архив успешно разархивирован.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import os

# Определение класса блока кодировщика
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        return self.block(x)

# Определение класса блока декодера
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.block(x)

# Определение архитектуры нейронной сети
class LungMaskGenerator(nn.Module):
    def __init__(self):
        super(LungMaskGenerator, self).__init__()
        self.encoder = nn.Sequential(
            EncoderBlock(3, 64),
            EncoderBlock(64, 128),
            EncoderBlock(128, 256),
            EncoderBlock(256, 512),
            EncoderBlock(512, 1024)
        )
        self.decoder = nn.Sequential(
            DecoderBlock(1024, 512),
            DecoderBlock(512, 256),
            DecoderBlock(256, 128),
            DecoderBlock(128, 64),
            DecoderBlock(64, 32),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# Пользовательский класс для загрузки данных
class LungDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.images[idx])
        mask_name = os.path.join(self.mask_dir, self.images[idx])
        image = Image.open(img_name).convert("RGB")
        mask = Image.open(mask_name).convert("L")  # Бинарная маска, поэтому используем режим "L"

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# Путь к папкам с изображениями и масками
train_image_dir = 'data/train_images/'
train_mask_dir = 'data/train_lung_masks/'

# Создание экземпляров DataLoader и нейронной сети
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = LungDataset(train_image_dir, train_mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

model = LungMaskGenerator()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [21]:
model = LungMaskGenerator().to(device)
c3 = torch.load('lung_mask_generatorV410.pth', map_location=device)
model.load_state_dict(c3)

<All keys matched successfully>

In [ ]:
# Обучение модели
num_epochs = 20
start_epochs = 11

for epoch in range(11, num_epochs + 1):
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(tqdm(dataloader)):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

    # Сохранение модели
    torch.save(model.state_dict(), f'/content/drive/MyDrive/lung_mask_generatorV4{epoch + 1}.pth')

 34%|███▎      | 2262/6750 [04:29<08:55,  8.38it/s]